In [13]:
import sys,os

#sys.path.append('/home/arash/VRdataCleaning/DeepSurv/')

import importlib
import deepsurv
from statsmodels.stats.outliers_influence import variance_inflation_factor    
import argparse
import uuid
import pickle
import json
import time
import numpy as np
import pandas as pd

import lasagne
import optunity

import logging
from logging import handlers
from sklearn.model_selection import train_test_split

importlib.reload(deepsurv)

from deepsurv import deep_surv, utils

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger
from eli5.permutation_importance import get_score_importances
import shap  # package used to calculate Shap values
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Load data: 
#### xtrain: 80% of all data, devided into xtrainsub & xvalsub (80 and 20%) 
#### xtest: 20% of all data

In [25]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'rb') as f:
    xtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrain', 'rb') as f:
    ytrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtest', 'rb') as f:
    xtest=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytest', 'rb') as f:
    ytest=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrainsub', 'rb') as f:
    xtrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrainsub', 'rb') as f:
    ytrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xvalsub', 'rb') as f:
    xvalsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/yvalsub', 'rb') as f:
    yvalsub=pickle.load(f) 

In [26]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

In [27]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

In [28]:
def load_logger(logdir):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    # Print to Stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # Print to Log file
    fh = logging.FileHandler(os.path.join(logdir, 'log_' + str(uuid.uuid4())))
    fh.setFormatter(format)
    logger.addHandler(fh)

    return logger

def format_to_deepsurv(x, y):
    return {
        'x': x,
        'e': y[:,0].astype(np.int32),
        't': y[:,1].astype(np.float32)
    }

In [29]:
def load_box_constraints(file):
    with open(file, 'rb') as fp:
        return json.loads(fp.read())

In [30]:
def save_call_log(file, call_log):
    with open(file, 'wb') as fp:
        pickle.dump(call_log, fp)

def get_objective_function(num_epochs, logdir, update_fn = lasagne.updates.sgd):
    '''
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to 
    use as hyper-parameters.

    The objective function runs a DeepSurv model on the training data and evaluates it against the
    a validation set. The result of the function call is the validation concordance index 
    (which Optunity tries to optimize)
    '''
    def format_to_deepsurv(x, y):
        return {
            'x': x,
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }

    def get_hyperparams(x,params):
        hyperparams = {
            'batch_norm': False,
            'activation': 'rectify',
            'standardize': True
        }

        if 'num_layers' in params and 'num_nodes' in params:
            params['hidden_layers_sizes'] = [int(params['num_nodes'])] * int(params['num_layers'])
            del params['num_layers']
            del params['num_nodes']

        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
        
        if 'n_in' in params:
            params['n_in']= int(np.floor(params['n_in']))
        
            
            

        hyperparams.update(params)
        return hyperparams

    def train_deepsurv(x_train,y_train,x_test,y_test,
        **kwargs):
        hyperparams = get_hyperparams(x_train,kwargs)
        x_train=x_train[:,0:hyperparams['n_in']]
        x_test=x_test[:,0:hyperparams['n_in']]
     
        
        # Standardize the datasets
        train_mean = x_train.mean(axis = 0)
        train_std = x_train.std(axis = 0)

        x_train = (x_train - train_mean) / train_std
        x_test = (x_test - train_mean) / train_std

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)

        

        # Set up Tensorboard loggers
        model_id = str(hash(str(hyperparams)))
        run_id = model_id + '_' + str(uuid.uuid4())
        logger = TensorboardLogger('hyperparam_search', 
            os.path.join(logdir,"tensor_logs", model_id, run_id))

        network = deep_surv.DeepSurv(**hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs, logger=logger, 
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        main_logger.info('Run id: %s | %s | C-Index: %f | Train Loss %f' % (run_id, str(hyperparams), result, metrics['loss'][-1][1]))
        return result

    return train_deepsurv


## Find the optimal hyper-parameters using training data and save them in opt_params


In [31]:
NUM_EPOCHS = 100
NUM_FOLDS = 8
logdir='/home/arash/ProjectVR/logs'


In [32]:
hyperparams = {
    'batch_norm': False,
    'activation': 'rectify',
    'standardize': True
}

In [33]:
if 'learning_rate' in hyperparams:
    hyperparams['learning_rate'] = 10 ** hyperparams['learning_rate']
 

In [34]:
main_logger = load_logger(logdir)


#    main_logger.debug('Loading dataset: ' + args.dataset)
box_constraints = load_box_constraints('/home/arash/ProjectVR/box_constraints.0.json')

In [35]:
opt_fxn = get_objective_function(NUM_EPOCHS, logdir, 
                                 utils.get_optimizer_from_str('adam'))

opt_fxn = optunity.cross_validated(x=xtrain, y=ytrain, num_folds=NUM_FOLDS)(opt_fxn)


opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=40,
        solver_name='sobol',
        **box_constraints)

KeyError: 'n_in'

In [50]:
opt_params

{'lr_decay': 0.00055607421875,
 'learning_rate': -3.290703125,
 'num_nodes': 50.3013671875,
 'num_layers': 3.02787109375,
 'momentum': 0.8054541015624999,
 'L2_reg': 3.5682705078125,
 'dropout': 0.1079609375,
 'n_in': 11.0484697265625}

## train a network based on opt_params on training data and save the model and weights (previous sections used cross-validation to find hyper-parameters, weights could not be saved)

In [ ]:
NUM_EPOCHS=4000
hyperparams = {
    'batch_norm': True,
    'activation': 'rectify',
    'standardize': False,
    'n_in':xtrain.shape[1]
}

hyperparams.update(opt_params)

if 'num_layers' in hyperparams and 'num_nodes' in hyperparams:
    hyperparams['hidden_layers_sizes'] = [int(hyperparams['num_nodes'])] * int(hyperparams['num_layers'])
    del hyperparams['num_layers']
    del hyperparams['num_nodes']

if 'learning_rate' in hyperparams:
    hyperparams['learning_rate'] = 10 ** hyperparams['learning_rate']
 

In [ ]:
### Standardizing
#trainsub_mean = xtrainsub.mean(axis = 0)
#trainsub_std = xtrainsub.std(axis = 0)
#xtrainsub = (xtrainsub - trainsub_mean) / trainsub_std

trainsub_data=format_to_deepsurv(xtrainsub, ytrainsub)

#valsub_mean = xvalsub.mean(axis = 0)
#valsub_std = xvalsub.std(axis = 0)
#xvalsub = (xvalsub - valsub_mean) / valsub_std

valsub_data=format_to_deepsurv(xvalsub, yvalsub)


network = deep_surv.DeepSurv(**hyperparams)

In [ ]:
metrics = network.train(trainsub_data,valsub_data, n_epochs = NUM_EPOCHS, update_fn = lasagne.updates.sgd, verbose = True)

In [ ]:
network.get_concordance_index(xvalsub,yvalsub[:,1],yvalsub[:,0])

## apply the trained network on test set and check the feature importance for generalization using  [permutation importance](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html)


In [ ]:
colnames=test.iloc[:,1:-1].columns
def score(X, y):
    X=pd.DataFrame(X)
    X.columns=colnames
    data=X
    data['Wait Time (s)']=y.values
    data['E']=1
    test = dataframe_to_deepsurv_ds(data, event_col = 'E', time_col= 'Wait Time (s)')
    xtest=test['x']
    xtest = (xtest - trainsub_mean) / trainsub_std              
    etest=test['e']
    ttest=test['t']
    ytest = np.column_stack((etest, ttest))
    valid_data = format_to_deepsurv(xtest, ytest)               #fix: network is trained again on test data
    ci=network.get_concordance_index(**valid_data)
    return ci


st=time.time()

base_score, score_decreases = get_score_importances(score, test.iloc[:,1:-1].values, test.iloc[:,0])
                                                    #,columns_to_shuffle=range(0,2))
f=time.time()

feature_importances_mean = np.mean(score_decreases, axis=0)
feature_importances_std = np.std(score_decreases, axis=0)

perimportance=pd.DataFrame(data=feature_importances_mean,index=colnames,columns=['mean'])
perimportance['std']=feature_importances_std



perimportance=perimportance.sort_values(by=['mean'])



def plot_feature_importances(imp):
    plt.figure(figsize=(16,12))
    n_features = imp.shape[0]
    plt.barh(range(n_features), perimportance.iloc[:,0].values, align='center')
    plt.yticks(np.arange(n_features), colnames)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances(perimportance)
#plt.savefig('Decision Tree feature_importance')

In [ ]:
NCtrain.iloc[:,1:-1].columns

## Using Shap for feature importance

In [ ]:
#using train set for interpretability
dfxtrainsub=pd.DataFrame(xtrain)
dfxtrainsub.columns=NCtrain.iloc[:,1:-1].columns

#using zeros as background data
backgrounddata=pd.DataFrame(np.zeros(32)).T         
backgrounddata.columns=NCtrain.iloc[:,1:-1].columns

explainer = shap.KernelExplainer(network.predict_risk,backgrounddata)
shap_values = explainer.shap_values(dfxtrainsub)

In [ ]:
rework=pd.DataFrame(shap_values[0])
rework.columns=features

In [ ]:
reworkclean=rework.drop(columns=['One Way Road', 'Two way with Median Road', 'Night Time', 'Age between 18 - 24',
    'Employed Person',
       'Student', 'Owns a Driving License', 'Main Mode: Car',
      'Main Mode: Walking', 'Sometimes Walks to Work',
       'Always Walks to Work', 'Sometimes Walks for shopping',
       'Speed Limit is Low', 'Speed Limit is high',
       'Minimum Gap is Low',
       'Mean Arrival Rate is Low', 'Does not own a car',
       'Has more than one car'])

In [ ]:
reworkclean.columns

In [ ]:
len(reworkclean.values)

In [ ]:
features=['Snowy Weather', 'Age between 25 - 29', 'Age between 30 - 39',
       'Gender: Female', 'Main Mode: Public Transit',
       'Always Walks for shopping', 'Has VR experinece', 'Lane Width is short',
       'Lane Width is long', 'Minimum Gap is High',
       'Mean Arrival Rate is High', 'Fully Automated Condition',
       'Mixed Automated Condition', 'Age is over 50']

In [ ]:
xx=shap.summary_plot(reworkclean.values, features,max_display=32)

In [ ]:
shap.summary_plot(reworkclean.values, features,max_display=32,show=False)

In [ ]:
import matplotlib.pyplot as pl
shap.summary_plot(reworkclean.values, features,max_display=32,show=False)
pl.savefig('shap.png',bbox_inches='tight')


In [ ]:
shap_df=pd.DataFrame(shap_values[0])
shap_df.columns=NCtrain.iloc[:,1:-1].columns

mean=shap_df[shap_df!=0].mean()              #mean of shap values for each feature (non-zeros only

nonzeros=shap_df.astype(bool).sum(axis=0)        #count of non-zeros shap values for each feature

std=shap_df[shap_df!=0].std()                #standard deviation of shap values for each feature

featuresshap=pd.DataFrame()

featuresshap['count1']=nonzeros
featuresshap['bg0mean']=mean                 #bg0: zeros set as background data

featuresshap['bg0std']=std

featuresshap['bg0absmean']=abs(mean)

featuresshap.sort_values(by=['bg0absmean'])

In [ ]:
#using ones as background data
backgrounddata2=pd.DataFrame(np.ones(42)).T         
backgrounddata2.columns=NCtrain.iloc[:,1:-1].columns

explainer2 = shap.KernelExplainer(network.predict_risk,backgrounddata2)
shap_values2 = explainer2.shap_values(dfxtrainsub)





In [ ]:
shap.summary_plot(shap_values2[0], dfxtrainsub,max_display=42)

In [ ]:
shap_values2=pd.DataFrame(shap_values2[0])
shap_values2.columns=NCtrain.iloc[:,1:-1].columns

mean2=shap_values2[shap_values2!=0].mean()              #mean of shap values for each feature (non-zeros only

std2=shap_values2[shap_values2!=0].std()                #standard deviation of shap values for each feature

featuresshap['bg1mean']=mean2                           #bg1: ones set as background data

featuresshap['bg1std']=std2

featuresshap['bg1absmean']=abs(mean2)

In [ ]:
featuresshap











